In [ ]:
pip install autoviz

In [ ]:
pip install xlrd

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
df_train = pd.read_csv('../input/tabular-playground-series-oct-2021/train.csv')
df_test  = pd.read_csv('../input/tabular-playground-series-oct-2021/test.csv')
ss       = pd.read_csv('../input/tabular-playground-series-oct-2021/sample_submission.csv')

In [ ]:
df_train.head()

In [ ]:
df_train.nunique().to_frame()

In [ ]:
df_train.describe()

# AutoViz

In [ ]:
from autoviz.AutoViz_Class import AutoViz_Class

AV = AutoViz_Class()

In [ ]:
filename = "../input/tabular-playground-series-oct-2021/train.csv"
sep = ","
dft = AV.AutoViz(
    filename,
    sep=",",
    depVar="target",
    dfte=None,
    header=0,
    verbose=0,
    lowess=False,
    chart_format="svg",
    max_rows_analyzed=150000,
    max_cols_analyzed=30,
)

# HistGradientBoost

In [ ]:
X = df_train.drop(columns=['id','target']).copy()
y = df_train['target'].copy()
X_test = df_test.drop(columns=['id']).copy()

In [ ]:
params = {'learning_rate': 0.7,
          'max_iter': 500,
          'max_leaf_nodes': 5,
          'max_depth': 7,
          'min_samples_leaf': 97,
          'l2_regularization': 0.012598398890369964}

In [ ]:
kf = StratifiedKFold(n_splits = 5, shuffle = True, random_state=2021)

preds = []
scores = []

for fold, (idx_train, idx_valid) in enumerate(kf.split(X, y)):
    X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
    X_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]
    
    model = HistGradientBoostingClassifier(**params,
                                           scoring='roc_auc')
    model.fit(X_train,y_train)
    
    pred_valid = model.predict_proba(X_valid)[:,1]
    fpr, tpr, _ = roc_curve(y_valid, pred_valid)
    score = auc(fpr, tpr)
    scores.append(score)
    
    print(f"Fold: {fold + 1} Score: {score}")
    print('::'*25)
    
    test_preds = model.predict_proba(X_test)[:,1]
    preds.append(test_preds)
    
print(f"Overall Validation Score: {np.mean(scores)}")

In [ ]:
predictions = np.mean(np.column_stack(preds),axis=1)
ss['target'] = predictions
ss.to_csv('./gbhist.csv', index=False)
ss.head()